In [1]:
from langchain import PromptTemplate

no_input_prompt = PromptTemplate(input_variables=[], template="请给我讲个笑话")
no_input_prompt.format()

'请给我讲个笑话'

In [2]:
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="请给我讲个{adjective}的笑话.")
one_input_prompt.format(adjective="有趣")

'请给我讲个有趣的笑话.'

In [3]:
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="请给我讲个关于{content}的{adjective}的笑话."
)
multiple_input_prompt.format(adjective="有趣", content="猪八戒")

'请给我讲个关于猪八戒的有趣的笑话.'

In [4]:
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qw_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

# qwen 兼容 openai的接口
qw_llm_openai = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [5]:
print(qw_llm_openai.invoke(multiple_input_prompt.format(adjective="灰色幽默", content="猪八戒")))

content='猪八戒，又名沙和尚，是《西游记》中的一个重要角色。他的形象非常滑稽，经常因为自己的愚蠢和不理智而让人捧腹大笑。\n\n有一天，唐僧师徒四人正在行进中，突然遇到了一只巨大的怪兽。他们决定使用孙悟空的金箍棒来对付这只怪兽。然而，当他们将金箍棒扔向怪兽时，奇怪的事情发生了：金箍棒变成了一个鸡蛋！\n\n这时，猪八戒急忙跑过来，大声喊道：“师傅，我们该怎么办？”他的话音刚落，金箍棒再次变成了一只小猪！\n\n看到这个情景，大家忍不住笑了起来。猪八戒也感到非常尴尬，但他并没有放弃，继续安慰大家说：“没关系，只要我们齐心协力，就一定能够战胜这只怪兽！”于是，他们一起合作，最终成功地打败了怪兽。\n\n这个故事告诉我们，即使是最聪明的人，在面对困难时也会犯错。但是，只要有团队精神和勇气，就能够克服一切困难！' response_metadata={'finish_reason': 'stop'} id='run-d77f539b-d174-4673-aa3b-5aad52b14337-0'


In [7]:
prompt = """The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

User: Where is New York?
AI: """

print(qw_llm_openai.invoke(prompt))

content="Hey there! You're looking for New York, huh? Well, you can't go wrong with the Big Apple. It's where it's at, baby!" response_metadata={'finish_reason': 'stop'} id='run-c2530ceb-00e9-478a-b21c-93c12cb64630-0'


In [8]:
examples = [
    {
        "query": "What sports do you play?",
        "answer": "Soccer, I play."
    }, {
        "query": "Where is this?",
        "answer": "My home, this is."
    }, {
        "query": "Are you leaving now?",
        "answer": "Stay and help you, I will."
    }, {
        "query": "How are you feeling?",
        "answer": "Great I feel."
    }
]

In [9]:
example_template = """
Human: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

Here are some examples:
"""

suffix = """
Human: {query}
AI: """

from langchain import FewShotPromptTemplate

# prefix：prompt前缀
# suffix：prompt后缀，用户输入
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

print(few_shot_prompt_template.format(query="Where is New York?"))

The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

Here are some examples:



Human: What sports do you play?
AI: Soccer, I play.



Human: Where is this?
AI: My home, this is.



Human: Are you leaving now?
AI: Stay and help you, I will.



Human: How are you feeling?
AI: Great I feel.



Human: Where is New York?
AI: 


In [10]:
print(qw_llm_openai.invoke(few_shot_prompt_template.format(query="Where is New York?")))

content="New York City, the Big Apple, it's where the action is!" response_metadata={'finish_reason': 'stop'} id='run-3b48be72-c84f-4be1-987c-6b1e75710804-0'


In [11]:
#  根据长度选择示例。
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_template = """
Human: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# example_selector 根据max_length来确定需要多少例子
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=30
)

In [12]:
few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)
print(few_shot_prompt_template.format(query="Where is New York?"))

The following are exerpts from conversations with an AI
assistant. The assistant is speaking like a famous movie character.

Here are some examples:


Human: What sports do you play?
AI: Soccer, I play.


Human: Where is this?
AI: My home, this is.


Human: Where is New York?
AI: 


In [13]:
print(qw_llm_openai.invoke(few_shot_prompt_template.format(query="Where is New York?")))

content="It's in the United States, on the east coast, near Canada and Canada." response_metadata={'finish_reason': 'stop'} id='run-dffa2c3e-a4cf-42fe-9747-d364e0bdd538-0'
